In [22]:
from tensorflow.keras.models import load_model


from google.colab import drive
drive.mount('/content/drive')


model = load_model('/content/drive/MyDrive/spl project/your_model_name.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
import numpy as np
import cv2
# IMG_SIZE should match the size used during training
IMG_SIZE = 128
# Updated image path: Assuming the image is in the current working directory
image_path = "/content/d3.jpg"
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print(f"Error: Could not load image from {image_path}. Check the path and file permissions.")
else:
    resized_array = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    resized_array = np.expand_dims(resized_array, axis=0)
    prediction = model.predict(resized_array)
    if prediction[0] == 0:
        print("Active")
    else:
        print("Drowsy")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 332ms/step
Drowsy


In [12]:
import os
import cv2
import numpy as np


os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'


IMG_SIZE = 128

# Path to the input video
video_path = "/content/drive/MyDrive/spl project/6687849-hd_1080_1920_25fps.mp4"

# Open the video file
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print(f"❌ Error: Could not open video file {video_path}. Check the path and file permissions.")
else:
    fatigue_frames = 0
    active_frames = 0
    frame_count = 0
    max_frames = 200  # Number of frames to process (adjust as needed)

    while cap.isOpened() and frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break

        # Resize and preprocess the frame
        resized_frame = cv2.resize(frame, (IMG_SIZE, IMG_SIZE))
        resized_frame = resized_frame / 255.0  # Normalize pixel values
        resized_frame = np.expand_dims(resized_frame, axis=0)

        # Predict without verbose output
        prediction = model.predict(resized_frame, verbose=0)
        pred_prob = prediction[0][0]

        # Frame-level prediction
        if pred_prob < 0.5:
            print(f"Frame {frame_count}: ✅ Active")
            active_frames += 1
        else:
            print(f"Frame {frame_count}: 😴 Fatigue")
            fatigue_frames += 1

        frame_count += 1

    cap.release()

    # Final results
    print("\n🎯 Summary:")
    print(f"Total frames processed: {frame_count}")
    print(f"✅ Active frames: {active_frames}")
    print(f"😴 Fatigue frames: {fatigue_frames}")

    if fatigue_frames > active_frames:
        print("\n🔔 Final Result: Drowsiness (Fatigue) Detected!")
    else:
        print("\n✅ Final Result: Active / No Drowsiness Detected.")


Frame 0: ✅ Active
Frame 1: ✅ Active
Frame 2: ✅ Active
Frame 3: ✅ Active
Frame 4: ✅ Active
Frame 5: ✅ Active
Frame 6: ✅ Active
Frame 7: ✅ Active
Frame 8: ✅ Active
Frame 9: ✅ Active
Frame 10: ✅ Active
Frame 11: ✅ Active
Frame 12: ✅ Active
Frame 13: ✅ Active
Frame 14: ✅ Active
Frame 15: ✅ Active
Frame 16: ✅ Active
Frame 17: ✅ Active
Frame 18: ✅ Active
Frame 19: ✅ Active
Frame 20: ✅ Active
Frame 21: ✅ Active
Frame 22: ✅ Active
Frame 23: ✅ Active
Frame 24: ✅ Active
Frame 25: ✅ Active
Frame 26: ✅ Active
Frame 27: ✅ Active
Frame 28: ✅ Active
Frame 29: ✅ Active
Frame 30: ✅ Active
Frame 31: ✅ Active
Frame 32: ✅ Active
Frame 33: ✅ Active
Frame 34: ✅ Active
Frame 35: ✅ Active
Frame 36: ✅ Active
Frame 37: ✅ Active
Frame 38: 😴 Fatigue
Frame 39: 😴 Fatigue
Frame 40: 😴 Fatigue
Frame 41: 😴 Fatigue
Frame 42: 😴 Fatigue
Frame 43: 😴 Fatigue
Frame 44: ✅ Active
Frame 45: ✅ Active
Frame 46: ✅ Active
Frame 47: 😴 Fatigue
Frame 48: 😴 Fatigue
Frame 49: 😴 Fatigue
Frame 50: 😴 Fatigue
Frame 51: 😴 Fatigue
Frame 52: 😴

In [2]:
# Import dependencies
from IPython.display import Javascript, display
from google.colab.output import eval_js
from base64 import b64decode
import cv2
import numpy as np
from PIL import Image
import io

# Webcam capture function
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
        async function takePhoto(quality) {
            const div = document.createElement('div');
            const capture = document.createElement('button');
            capture.textContent = '📸 Capture';
            div.appendChild(capture);

            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(div);
            div.appendChild(video);
            video.srcObject = stream;
            await video.play();

            google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
            await new Promise((resolve) => capture.onclick = resolve);

            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);
            stream.getTracks().forEach(track => track.stop());
            div.remove();

            const dataUrl = canvas.toDataURL('image/jpeg', quality);
            const bytes = await fetch(dataUrl).then(res => res.arrayBuffer());
            return Array.from(new Uint8Array(bytes));
        }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    image_bytes = bytes(data)
    image = Image.open(io.BytesIO(image_bytes))
    image.save(filename)
    return filename


In [21]:
IMG_SIZE = 128

def preprocess_and_predict(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255.0
    img = np.expand_dims(img, axis=0)

    pred = model.predict(img)
    probability = pred[0][0]  # The output probability for drowsiness

    label = "active" if probability > 0.5 else "drowsy"

    return label, probability

label, probability = preprocess_and_predict(filename)
print("Prediction:", label)
print("Prediction Probability:", probability)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Prediction: active
Prediction Probability: 0.54315674


In [20]:
filename = take_photo()
print("Saved to:", filename)


<IPython.core.display.Javascript object>

Saved to: photo.jpg
